In [8]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import xgboost as xgb
from lifelines import KaplanMeierFitter
#from sklearn.metrics import mean_absolute_error, mean_squared_error
#from sklearn.model_selection import train_test_split

In [9]:
from pathlib import Path
path_folder = Path("/Users/keithlowton/Desktop/Ks/Python code/Kaggle/Telco Customer Churn")
print(path_folder.exists())
print(path_folder)

True
/Users/keithlowton/Desktop/Ks/Python code/Kaggle/Telco Customer Churn


In [10]:
churn_data = pd.read_csv(os.path.join(path_folder, "data/churn_data.csv"), sep=",")
print(churn_data.shape)

# Converting Yes to 1 and No to 0 in Churn column
# churn_data['Churn'] = churn_data['Churn'].map({'Yes': 1, 'No': 0})

# Subseting Of The Columns To Train The Model
features_to_drop = ["customerID", "PhoneService"]

# Engineering Numeric Columns For Cox Model
churn_surv_data = churn_data.drop(features_to_drop, axis=1).copy()

# Convert Variable To Integers
churn_surv_data["TotalCharges"] = pd.to_numeric(churn_surv_data["TotalCharges"], errors="coerce")
churn_surv_data["TotalCharges"].fillna(0, inplace=True)

# Encoded Variables As 'Yes' Or 'No'
binary_features = [
    "PaperlessBilling",
]

for feat in binary_features:
    churn_surv_data[feat] = churn_surv_data[feat] == "Yes"

# One Hot Encoding For Categorical Features
ohe_features = ["Contract", "PaymentMethod"]
churn_surv_data = pd.get_dummies(churn_surv_data, drop_first=True, columns=ohe_features)

churn_surv_data = churn_surv_data[(churn_surv_data.tenure > 0)]
churn_surv_data.head()

(7043, 9)


AttributeError: 'Index' object has no attribute '_format_flat'

   tenure  PaperlessBilling  MonthlyCharges  TotalCharges Churn  \
0       1              True           29.85         29.85    No   
1      34             False           56.95       1889.50    No   
2       2              True           53.85        108.15   Yes   
3      45             False           42.30       1840.75    No   
4       2              True           70.70        151.65   Yes   

   Contract_One year  Contract_Two year  \
0              False              False   
1               True              False   
2              False              False   
3               True              False   
4              False              False   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                  False                            True   
1                                  False                           False   
2                                  False                           False   
3                                 

In [11]:
# Time to event data of censored and event data
durations =  churn_surv_data['tenure']
# It has the churned (1) and censored is (0)
event_observed =  churn_surv_data['Churn']

# create a kmf object as km
# instantiate the class to create an object
km = KaplanMeierFitter()

# Fit the data into the model
km = km.fit(durations, event_observed, label ='Kaplan Meier Estimate')

# Create an estimate
# km.plot()


# Kaplan-Meijer estimate of the survival function for contract type (Month-to-month)
ax = plt.subplot(111)

# instantiate the class to create an object
kmf = KaplanMeierFitter()

# time to event
T = churn_data['tenure']
# event occurred or censored
E = churn_data['Churn']

# Create the cohorts from the 'Contract' column
con_type = churn_data['Contract']
m_m = (groups == 'Month-to-month')

# Fit the cohort Month-to-month
kmf.fit(T[m_m], E[m_m], label = 'Month-to-month')
kmf.plot(ax = ax, color = 'purple')

plt.ylim(0, 1);
plt.title("Survival Curves Based On Contract");
plt.xlabel('Time since subscritption start')
plt.ylabel('Probablity still a customer')

image = 'Survival Curves Based On Contract.png'
plt.savefig(image, dpi = 600)
plt.show()

/Users/keithlowton/anaconda3/envs/Survival_analysis/lib/python3.10/site-packages/lifelines/utils/__init__.py:1185: UserWarning: Attempting to convert an unexpected datatype 'object' to float. Suggestion: 1) use `lifelines.utils.datetimes_to_durations` to do conversions or 2) manually convert to floats/booleans.
  warnings.warn(warning_text, UserWarning)


TypeError: Wrong dtype 'object'.

In [ ]:
from lifelines import CoxPHFitter

# Applying Cox model with 0.1 penalty
cph = CoxPHFitter(penalizer = 0.1)
# Using Cox Proportional Hazards model
# Instantiate the class to create a cph object
cph = CoxPHFitter()
# Fit the data to train the model
cph.fit(churn_surv_data, 'tenure', event_col = 'Churn')
# Have a look at the significance of the features
cph.print_summary()

# Removing var with  p-value > 0.05
# There was variable with p-value 0.05 which is considered as insignificant. So, we removed those variables from data which has p-value greater than 0.05
churn_surv_data_2 = churn_surv_data.drop(["SeniorCitizen", "Dependents", "PaperlessBilling", "PaymentMethod_Credit card (automatic)"], axis = 1)

# Again applying Cox model with 0.1 penalty on updated data
cph_2 = CoxPHFitter(penalizer = 0.1)
cph_2.fit(churn_surv_data_2, 'tenure', event_col = 'Churn')
cph_2.print_summary()
<lifelines.CoxPHFitter: fitted with 7032 observations, 5163 censored>
      duration col = 'tenure'
         event col = 'Churn'
         penalizer = 0.1
number of subjects = 7032
  number of events = 1869
partial log-likelihood = -12694.40
  time fit was run = 2019-11-11 13:37:59 UTC

---
                                coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
Partner                        -0.23      0.80      0.05           -0.32           -0.13                0.72                0.88
OnlineSecurity                 -0.40      0.67      0.07           -0.53           -0.27                0.59                0.76
OnlineBackup                   -0.30      0.74      0.06           -0.41           -0.19                0.66                0.83
DeviceProtection               -0.17      0.84      0.06           -0.28           -0.06                0.76                0.94
TechSupport                    -0.29      0.75      0.07           -0.42           -0.16                0.66                0.85
StreamingTV                    -0.27      0.76      0.06           -0.39           -0.15                0.68                0.86
StreamingMovies                -0.27      0.77      0.06           -0.38           -0.15                0.68                0.86
MonthlyCharges                  0.07      1.07      0.00            0.06            0.07                1.07                1.08
TotalCharges                   -0.00      1.00      0.00           -0.00           -0.00                1.00                1.00
MultipleLines_No phone service  0.70      2.01      0.11            0.48            0.92                1.61                2.52
MultipleLines_Yes              -0.19      0.82      0.05           -0.30           -0.09                0.74                0.92
Contract_One year              -1.42      0.24      0.10           -1.62           -1.23                0.20                0.29
Contract_Two year              -4.08      0.02      0.19           -4.46           -3.70                0.01                0.02
PaymentMethod_Electronic check  0.39      1.48      0.06            0.27            0.50                1.32                1.66
PaymentMethod_Mailed check      0.51      1.66      0.08            0.36            0.65                1.43                1.92

                                    z      p  -log2(p)
Partner                         -4.46 <0.005     16.88
OnlineSecurity                  -6.03 <0.005     29.20
OnlineBackup                    -5.34 <0.005     23.37
DeviceProtection                -3.02 <0.005      8.63
TechSupport                     -4.42 <0.005     16.66
StreamingTV                     -4.55 <0.005     17.49
StreamingMovies                 -4.46 <0.005     16.93
MonthlyCharges                  27.55 <0.005    552.74
TotalCharges                   -40.13 <0.005       inf
MultipleLines_No phone service   6.12 <0.005     29.95
MultipleLines_Yes               -3.59 <0.005     11.55
Contract_One year              -14.05 <0.005    146.61
Contract_Two year              -20.98 <0.005    322.27
PaymentMethod_Electronic check   6.65 <0.005     35.00
PaymentMethod_Mailed check       6.73 <0.005     35.74
---
Concordance = 0.93
Log-likelihood ratio test = 5917.28 on 15 df, -log2(p)=inf
view rawcox_ph_fitter.js hosted with ❤ by GitHub

In [ ]:
# Pediction
tr_rows1 = churn_surv_data_2.iloc[:, 2:]
pred_cxh = cph_2.predict_survival_function(tr_rows1)
# pred_cxh

# Filtered ~ 1 % of the data
filtered_pred_cxh = pred_cxh.loc[:, (pred_cxh.gt(0.588199) & pred_cxh.lt(0.999998)).all(0)]
filtered_pred_cxh.head()

            1	   3	       9         	10	       11	      12	      17        	24		57	...	      7040
1.0	   0.979182	0.984832	0.976538	0.978377	0.980788	0.976950	0.978191	0.981877	0.989895	0.985345
2.0	   0.957534	0.968966	0.952207	0.955911	0.960776	0.953036	0.955536	0.962977	0.979268	0.970007
3.0	   0.935872	0.952990	0.927932	0.933449	0.940716	0.929166	0.932890	0.944008	0.968510	0.954555
4.0	   0.914381	0.937041	0.903919	0.911186	0.920778	0.905543	0.910449	0.925133	0.957708	0.939119
5.0	   0.893148	0.921182	0.880266	0.889210	0.901045	0.882264	0.888302	0.906428	0.946906	0.923763
6.0	   0.872224	0.905453	0.857028	0.867574	0.881562	0.859381	0.866502	0.887938	0.936131	0.908522
7.0	   0.851639	0.889877	0.834236	0.846308	0.862359	0.836928	0.845080	0.869690	0.925399	0.893422
8.0	   0.831413	0.874473	0.811910	0.825432	0.843456	0.814923	0.824055	0.851705	0.914722	0.878478
9.0	   0.811557	0.859251	0.790062	0.804958	0.824865	0.793378	0.803440	0.833994	0.904110	0.863703
10.0   0.792081	0.844220	0.768697	0.784894	0.806594	0.772300	0.783242	0.816566	0.893569	0.849105

# Plot individual survival curves
p1 = ffiltered_pred_cxh.plot(legend=None)
plt.xlabel('Time since subscritption start')
plt.ylabel('Probablity still a customer')
# remove all the ticks (both axes), and tick labels on the Y axis
# plt.tick_params(top ='off', bottom='off', left='off', right='off', labelleft ='off', labelbottom='off')
# plt.yticks(np.arange(.4, 1, .1))
image = 'time_since_customer_cxh.png'
plt.savefig(image, dpi = 600)

from IPython.display import HTML
HTML('<a href ="{0}" download>Click here to download {0}</a>'.format(image))
view rawfiltered_pred_cxh.js hosted with ❤ by GitHub